In [489]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.mllib.recommendation import ALS as newALS

In [490]:
ratings_path = "~/Downloads/ml-100k/u.data"
df = pd.read_csv(ratings_path, sep = '\t').sort_values('timestamp')

In [491]:
#ratings_path = "~/Downloads/ml-1m/ratings.dat"
#df = pd.read_csv(ratings_path, sep = '::', names=['user_id', 'movie_id', 'rating', 'timestamp']).sort_values('timestamp')

In [492]:
df

,user_id,movie_id,rating,timestamp
214,259,255,4,874724710
83965,259,286,4,874724727
43027,259,298,4,874724754
21396,259,185,4,874724781
82655,259,173,4,874724843
...,...,...,...,...
46773,729,689,4,893286638
73008,729,313,3,893286638
46574,729,328,3,893286638
64312,729,748,4,893286638


In [493]:
len(df)

100000

In [494]:
squash_amount = 100

In [495]:
new_timestamp = [i // squash_amount for i in range(1, len(df)+1)]
new_timestamp

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [496]:
new_df = df.drop('timestamp', 1)
new_df['timestamp'] = new_timestamp

/var/folders/p5/zwk89rgd0cdf07_7k1wdqjgw0000gn/T/ipykernel_56430/405703479.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  new_df = df.drop('timestamp', 1)


In [497]:
new_df

,user_id,movie_id,rating,timestamp
214,259,255,4,0
83965,259,286,4,0
43027,259,298,4,0
21396,259,185,4,0
82655,259,173,4,0
...,...,...,...,...
46773,729,689,4,999
73008,729,313,3,999
46574,729,328,3,999
64312,729,748,4,999


In [498]:
def train_stream_test_split(df):
    columns = ["user_id", "movie_id", "rating", "timestamp"]
    als_data = []
    stream_data = []
    for user_id in df["user_id"].unique():
        user_df = df[df["user_id"] == user_id].sort_values("timestamp")
        count = 0
        for row in user_df.itertuples():
            data = [row.user_id, row.movie_id, row.rating, row.timestamp]
            if count < len(user_df) / 2:
                als_data.append(data)
            else:
                stream_data.append(data)
            count += 1
    als_df = pd.DataFrame(data=als_data, columns=columns).sort_values("timestamp")
    movie_list = set(als_df['movie_id'])
    stream_df = pd.DataFrame(data=stream_data, columns=columns).sort_values("timestamp")
    # Drop movies not seen before from stream
    stream_df = stream_df.drop(stream_df[stream_df['movie_id'].map(lambda x: x not in movie_list)].index)
    test_df = stream_df.copy()
    return als_df, stream_df, test_df

In [499]:
als_df, stream_df, test_df = train_stream_test_split(new_df)

In [510]:
spark.stop()

In [511]:
spark = SparkSession.builder.master('local').appName('als').getOrCreate()

In [512]:
als_df = als_df.drop("timestamp", 1)

/var/folders/p5/zwk89rgd0cdf07_7k1wdqjgw0000gn/T/ipykernel_56430/2193871712.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  als_df = als_df.drop("timestamp", 1)


KeyError: "['timestamp'] not found in axis"

In [513]:
spark_als_df = spark.createDataFrame(als_df) 
spark_stream_df = spark.createDataFrame(stream_df)

In [514]:
spark_als_df.toPandas()

,user_id,movie_id,rating
0,259,255,4
1,712,83,4
2,712,196,4
3,712,191,3
4,712,716,5
...,...,...,...
50235,683,1483,3
50236,683,22,4
50237,683,626,3
50238,683,62,4


In [504]:
als = ALS(
         userCol="user_id", 
         itemCol="movie_id",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop",
         rank=150,
         maxIter=10,
         regParam=.1
)
model=als.fit(spark_als_df)
predictions = model.transform(spark_stream_df)
evaluator = RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")
rmse = evaluator.evaluate(predictions)
rmse

In [515]:
new_model = newALS.train(ratings = spark_als_df.rdd, rank = 150, iterations=15, lambda_=.2)

In [518]:
new_model.userFeatures().toDF().toPandas()

,_1,_2
0,1,"[0.1569247841835022, -0.16008520126342773, 0.2..."
1,2,"[0.13312149047851562, -0.1589306890964508, 0.2..."
2,3,"[0.14130325615406036, 0.017957353964447975, 0...."
3,4,"[0.1787623018026352, -0.005274169612675905, 0...."
4,5,"[0.1612255573272705, -0.13689665496349335, 0.2..."
...,...,...
938,939,"[-0.07123947888612747, -0.19237439334392548, 0..."
939,940,"[0.0003264975093770772, -0.14734606444835663, ..."
940,941,"[0.14008139073848724, -0.16182909905910492, 0...."
941,942,"[0.06645748764276505, -0.14005815982818604, 0...."


In [519]:
user_features = new_model.userFeatures().toDF().toPandas()
user_features = user_features.rename(columns={'_1': 'id', '_2': 'features'})
user_features

,id,features
0,1,"[0.1569247841835022, -0.16008520126342773, 0.2..."
1,2,"[0.13312149047851562, -0.1589306890964508, 0.2..."
2,3,"[0.14130325615406036, 0.017957353964447975, 0...."
3,4,"[0.1787623018026352, -0.005274169612675905, 0...."
4,5,"[0.1612255573272705, -0.13689665496349335, 0.2..."
...,...,...
938,939,"[-0.07123947888612747, -0.19237439334392548, 0..."
939,940,"[0.0003264975093770772, -0.14734606444835663, ..."
940,941,"[0.14008139073848724, -0.16182909905910492, 0...."
941,942,"[0.06645748764276505, -0.14005815982818604, 0...."


In [520]:
product_features = new_model.productFeatures().toDF().toPandas()
product_features = product_features.rename(columns={'_1': 'id', '_2': 'features'})

22/02/22 17:35:21 WARN BlockManager: Task 50 already completed, not releasing lock for rdd_315_0


In [521]:
dir_path = "~/movielens/ralf/"

In [522]:
product_features.sort_values('id').to_csv(dir_path + 'movie_features.csv', index = False)

In [523]:
user_features.sort_values('id').to_csv(dir_path + 'user_features.csv', index = False)

In [525]:
stream_df = stream_df.sample(frac=1)
new_timestamp = [i // squash_amount for i in range(1, len(stream_df)+1)]
stream_df['timestamp'] = new_timestamp

In [526]:
stream_df

,user_id,movie_id,rating,timestamp
15583,843,141,4,0
25736,454,275,2,0
27865,854,8,5,0
47562,334,945,4,0
10380,299,606,4,0
...,...,...,...,...
13766,95,1230,1,494
9891,463,751,4,494
9513,331,268,5,494
19923,128,229,2,494


In [527]:
print(len(stream_df))

49442


In [528]:
stream_df.to_csv(dir_path + 'ratings.csv', index = False)